# AlphaFold 2 Interactive Notebook

This notebook demonstrates how to actually run AlphaFold 2 predictions using ColabFold.

**What you'll learn:**
- Install and configure ColabFold (AlphaFold 2 implementation)
- Run actual protein structure predictions
- Visualize 3D structures interactively
- Analyze confidence metrics (pLDDT and PAE)
- Compare multiple model predictions

**Note:** This notebook requires GPU for optimal performance. In Google Colab, select: Runtime → Change runtime type → GPU

## 1. Setup and Installation

First, we need to install ColabFold and its dependencies. This includes the AlphaFold 2 model weights and required packages.

In [ ]:
# Install ColabFold and dependencies
# This may take 5-10 minutes on first run
# Note: Use the official ColabFold installation method
import os
import sys

if not os.path.isfile("COLABFOLD_READY"):
    print("Installing ColabFold...")
    !pip install -q --no-warn-conflicts 'colabfold[alphafold-minus-jax] @ git+https://github.com/sokrypton/ColabFold'
    !ln -s /usr/local/lib/python3.*/dist-packages/colabfold colabfold 2>/dev/null || true
    !ln -s /usr/local/lib/python3.*/dist-packages/alphafold alphafold 2>/dev/null || true
    !touch COLABFOLD_READY
    print("✓ ColabFold installed")

# Install visualization tools
!pip install -q py3Dmol biopython matplotlib pandas

# Check GPU availability (essential for fast predictions)
try:
    import torch
    if torch.cuda.is_available():
        print(f"\n✓ GPU available: {torch.cuda.get_device_name(0)}")
        print(f"  Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")
    else:
        print("\n⚠ No GPU detected. Predictions will be very slow!")
        print("  In Google Colab: Runtime → Change runtime type → GPU")
except:
    print("\n⚠ PyTorch not available for GPU check, continuing anyway...")

## 2. Import Required Libraries

Import all necessary packages for prediction, visualization, and analysis.

In [ ]:
# Core libraries
import json
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import warnings

# Suppress deprecation warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
try:
    from Bio import BiopythonDeprecationWarning
    warnings.simplefilter(action='ignore', category=BiopythonDeprecationWarning)
except:
    pass

# BioPython for structure handling
from Bio.PDB import PDBParser

# Visualization
import py3Dmol

print("✓ All libraries imported successfully!")

## 3. Configure AlphaFold Environment

Set up directories and configure prediction parameters.

In [ ]:
# Create output directories
output_dir = Path("./predictions")
output_dir.mkdir(exist_ok=True)

# AlphaFold 2 prediction parameters (matching official ColabFold defaults)
config = {
    "num_models": 5,          # Number of model variants to run (1-5)
    "num_recycles": 3,         # Number of recycling iterations (default=3, multimer_v3=20)
    "num_relax": 1,            # Number of structures to relax with Amber (0=none, 1=top, 5=all)
    "use_templates": False,    # Don't use template structures
    "msa_mode": "mmseqs2_uniref_env",  # MSA generation method
    "model_type": "auto",      # auto, alphafold2_ptm, alphafold2_multimer_v3
    "max_msa": None,           # None=auto, or specify like "512:1024"
    "pair_mode": "unpaired_paired",  # For complexes: unpaired_paired, paired, unpaired
}

print("Configuration (matching official ColabFold):")
for key, value in config.items():
    print(f"  {key}: {value}")

print("\n💡 These parameters match the official ColabFold notebook defaults")
print(f"\n✓ Output directory: {output_dir.absolute()}")

## 4. Prepare Protein Sequence Input

Define your protein sequence(s) for structure prediction. You can use:
- Single protein sequences
- Protein complexes (multiple chains)
- Your own sequences (replace the demo sequence)

In [ ]:
# Demo protein: Insulin B chain (small example, ~30 residues)
# This will run quickly for demonstration purposes
protein_sequence = "FVNQHLCGSHLVEALYLVCGERGFFYTPKA"
protein_name = "insulin_b_chain"

# Alternative: Try your own protein!
# Uncomment and replace with your sequence:
# protein_sequence = "YOUR_AMINO_ACID_SEQUENCE_HERE"
# protein_name = "my_protein"

# Validate sequence (only standard amino acids)
valid_amino_acids = set("ACDEFGHIKLMNPQRSTVWY")
if not all(aa in valid_amino_acids for aa in protein_sequence.upper()):
    print("⚠ Warning: Sequence contains non-standard amino acids!")
else:
    print(f"✓ Valid sequence: {len(protein_sequence)} residues")

# Create FASTA file
fasta_path = output_dir / f"{protein_name}.fasta"
with open(fasta_path, 'w') as f:
    f.write(f">{protein_name}\n{protein_sequence}\n")

print(f"\n✓ FASTA file created: {fasta_path}")
print("\nSequence preview:")
print(f">{protein_name}")
print(f"{protein_sequence}")

## 5. Run AlphaFold 2 Prediction

**This is where the actual prediction happens!**

The prediction process:
1. Searches sequence databases for similar proteins (MSA generation)
2. Runs AlphaFold 2 neural network to predict 3D structure
3. Refines structure through recycling
4. Optionally relaxes geometry with Amber force field

⏱ **Time estimate:** 2-5 minutes for small proteins (~30-50 residues), 10-30 minutes for larger proteins (100-500 residues)

In [ ]:
# Run ColabFold prediction
# This executes the actual AlphaFold 2 model
print("Starting AlphaFold 2 prediction...")
print("=" * 60)

# Run using command line (matching official ColabFold notebook)
# The official notebook uses colabfold.batch.run() but command line works too
cmd = f"""colabfold_batch \
    {fasta_path} \
    {output_dir} \
    --num-models {config['num_models']} \
    --num-recycle {config['num_recycles']} \
    --num-relax {config['num_relax']} \
    --msa-mode {config['msa_mode']} \
    --model-type {config['model_type']} \
    --use-gpu-relax
"""

print(f"Command: {cmd}\n")
print("This may take:")
print("  - 2-5 min for small proteins (~30-50 residues)")
print("  - 10-30 min for typical proteins (100-300 residues)")
print("  - 30-60 min for large proteins (>500 residues)\n")

!{cmd}

print("\n" + "=" * 60)
print("✓ Prediction complete!")
print("\n💡 Output files match official ColabFold format:")
print("  - *_relaxed_rank_001_*.pdb (best model, Amber relaxed)")
print("  - *_unrelaxed_rank_001_*.pdb (best model, unrelaxed)")
print("  - *_scores_rank_001_*.json (confidence metrics)")
print("  - *_plddt.png (confidence plot)")
print("  - *_pae.png (predicted aligned error)")
print("  - *_coverage.png (MSA coverage plot)")
print(f"\nResults saved in: {output_dir}")

## 6. Load and Visualize Predicted Structure

Visualize the predicted 3D structure with interactive py3Dmol viewer. The structure is colored by pLDDT confidence:
- **Blue** = Very high confidence (>90)
- **Cyan** = High confidence (70-90)
- **Yellow** = Low confidence (50-70)
- **Orange/Red** = Very low confidence (<50)

In [ ]:
# Find the best predicted structure (ranked_0.pdb)
pdb_files = list(output_dir.glob("*_relaxed_rank_001_*.pdb"))
if not pdb_files:
    pdb_files = list(output_dir.glob("*_unrelaxed_rank_001_*.pdb"))

if not pdb_files:
    print("⚠ No PDB files found. Check if prediction completed successfully.")
else:
    best_pdb = str(pdb_files[0])
    print(f"Loading structure: {Path(best_pdb).name}\n")
    
    # Read PDB file
    with open(best_pdb, 'r') as f:
        pdb_data = f.read()
    
    # Create 3D viewer
    view = py3Dmol.view(width=800, height=600)
    view.addModel(pdb_data, 'pdb')
    
    # Color by pLDDT (B-factor column in PDB)
    view.setStyle({'cartoon': {'colorscheme': 'ssBfactor'}})
    view.zoomTo()
    
    print("Interactive 3D Structure Viewer:")
    print("- Drag to rotate")
    print("- Scroll to zoom")
    print("- Right-click drag to pan\n")
    
    view.show()

## 7. Analyze Confidence Metrics (pLDDT and PAE)

**pLDDT** (predicted Local Distance Difference Test): Per-residue confidence score (0-100)
- Higher is better
- >90 = very reliable
- 70-90 = generally good
- <70 = uncertain/disordered

**PAE** (Predicted Aligned Error): Uncertainty in relative positions between residues
- Lower is better
- Blue regions = confident
- Red regions = uncertain

In [ ]:
# Load pLDDT scores from PDB file (stored in B-factor column)
parser = PDBParser(QUIET=True)
structure = parser.get_structure('protein', best_pdb)
plddt_scores = [atom.bfactor for atom in structure.get_atoms() if atom.name == 'CA']

# Load PAE data from JSON
json_files = list(output_dir.glob("*_scores_rank_001_*.json"))
if json_files:
    with open(json_files[0], 'r') as f:
        scores_data = json.load(f)
    pae_matrix = np.array(scores_data['pae'])
else:
    pae_matrix = None

# Plot pLDDT scores
fig, axes = plt.subplots(1, 2, figsize=(16, 5))

# pLDDT line plot
ax = axes[0]
residues = list(range(1, len(plddt_scores) + 1))
ax.plot(residues, plddt_scores, linewidth=2, color='#1f77b4')
ax.axhline(y=90, color='green', linestyle='--', alpha=0.5, label='Very confident (>90)')
ax.axhline(y=70, color='orange', linestyle='--', alpha=0.5, label='Confident (>70)')
ax.axhline(y=50, color='red', linestyle='--', alpha=0.5, label='Low confidence (>50)')
ax.set_xlabel('Residue Number', fontsize=12)
ax.set_ylabel('pLDDT Score', fontsize=12)
ax.set_title('Per-Residue Confidence (pLDDT)', fontsize=14, fontweight='bold')
ax.set_ylim([0, 100])
ax.legend()
ax.grid(alpha=0.3)

# PAE heatmap
ax = axes[1]
if pae_matrix is not None:
    im = ax.imshow(pae_matrix, cmap='Blues_r', vmin=0, vmax=30)
    ax.set_xlabel('Scored Residue', fontsize=12)
    ax.set_ylabel('Aligned Residue', fontsize=12)
    ax.set_title('Predicted Aligned Error (PAE)', fontsize=14, fontweight='bold')
    plt.colorbar(im, ax=ax, label='Expected Error (Å)')
else:
    ax.text(0.5, 0.5, 'PAE data not available', ha='center', va='center', transform=ax.transAxes)
    ax.set_xticks([])
    ax.set_yticks([])

plt.tight_layout()
plt.show()

# Print summary statistics
print("\nConfidence Summary:")
print(f"  Mean pLDDT: {np.mean(plddt_scores):.1f}")
print(f"  Min pLDDT:  {np.min(plddt_scores):.1f}")
print(f"  Max pLDDT:  {np.max(plddt_scores):.1f}")
print(f"  Residues with pLDDT > 90: {np.sum(np.array(plddt_scores) > 90)} / {len(plddt_scores)}")
print(f"  Residues with pLDDT > 70: {np.sum(np.array(plddt_scores) > 70)} / {len(plddt_scores)}")

## 8. Compare Multiple Models

AlphaFold 2 generates multiple predictions (usually 5). Comparing them helps assess consistency.

In [ ]:
# Find all ranked models
all_models = sorted(output_dir.glob("*_rank_*_*.pdb"))

print(f"Found {len(all_models)} predicted models:\n")

# Compare confidence scores across models
model_scores = []
for model_path in all_models[:5]:  # Top 5 models
    parser = PDBParser(QUIET=True)
    structure = parser.get_structure('protein', str(model_path))
    plddt = [atom.bfactor for atom in structure.get_atoms() if atom.name == 'CA']
    mean_plddt = np.mean(plddt)
    model_scores.append((model_path.name, mean_plddt))
    print(f"  {model_path.name[:50]:50s}  Mean pLDDT: {mean_plddt:.2f}")

# Plot comparison
if len(model_scores) > 1:
    fig, ax = plt.subplots(figsize=(10, 5))
    models = [f"Model {i+1}" for i in range(len(model_scores))]
    scores = [score for _, score in model_scores]
    
    bars = ax.bar(models, scores, color='steelblue', alpha=0.7)
    ax.axhline(y=90, color='green', linestyle='--', alpha=0.5, label='Very confident')
    ax.axhline(y=70, color='orange', linestyle='--', alpha=0.5, label='Confident')
    ax.set_ylabel('Mean pLDDT Score', fontsize=12)
    ax.set_title('Model Comparison by Confidence', fontsize=14, fontweight='bold')
    ax.set_ylim([0, 100])
    ax.legend()
    ax.grid(axis='y', alpha=0.3)
    
    # Add value labels on bars
    for bar, score in zip(bars, scores):
        height = bar.get_height()
        ax.text(bar.get_x() + bar.get_width()/2., height + 1,
                f'{score:.1f}', ha='center', va='bottom', fontsize=10)
    
    plt.tight_layout()
    plt.show()
    
    print(f"\n✓ Best model: {model_scores[0][0][:50]}")
    print("  (Ranked by ColabFold based on pLDDT and pTM scores)")

## 9. Export and Save Results

Save your results for further analysis or publication.

In [ ]:
# Create summary report
summary = {
    "protein_name": protein_name,
    "sequence": protein_sequence,
    "sequence_length": len(protein_sequence),
    "prediction_parameters": config,
    "mean_plddt": float(np.mean(plddt_scores)),
    "min_plddt": float(np.min(plddt_scores)),
    "max_plddt": float(np.max(plddt_scores)),
    "high_confidence_residues": int(np.sum(np.array(plddt_scores) > 90)),
    "total_residues": len(plddt_scores),
}

# Save summary as JSON
summary_path = output_dir / f"{protein_name}_summary.json"
with open(summary_path, 'w') as f:
    json.dump(summary, f, indent=2)

print("✓ Summary saved to:", summary_path)
print("\nSummary:")
print(json.dumps(summary, indent=2))

# List all output files
print(f"\n\nAll generated files in {output_dir}:")
for file in sorted(output_dir.glob("*")):
    size_mb = file.stat().st_size / (1024 * 1024)
    print(f"  {file.name:60s} ({size_mb:.2f} MB)")

## 🎉 Congratulations!

You've successfully run AlphaFold 2 to predict a protein structure!

### Next Steps:
1. **Try your own protein**: Replace the demo sequence with your protein of interest
2. **Analyze domains**: Use pLDDT scores to identify structured vs. disordered regions
3. **Compare with experimental structures**: Download PDB files from RCSB and compare
4. **Visualize in other tools**: Open PDB files in PyMOL, ChimeraX, or VMD

### Tips for Better Predictions:
- Longer sequences (>100 residues) usually work better
- Proteins with many homologs give better MSA and higher confidence
- Low pLDDT regions may indicate intrinsically disordered regions (IDRs)
- Check PAE matrix for domain interactions in multi-domain proteins

### Resources:
- **AlphaFold 2 Paper**: [Nature (2021)](https://www.nature.com/articles/s41586-021-03819-2)
- **ColabFold Paper**: [Nature Methods (2022)](https://www.nature.com/articles/s41592-022-01488-1)
- **AlphaFold Database**: [alphafold.ebi.ac.uk](https://alphafold.ebi.ac.uk/)
- **RCSB PDB**: [www.rcsb.org](https://www.rcsb.org/)